## Spots that are always excluded:
##### Test spots
##### Spots marked as problematic
##### Suscpicious for analysis == 'Yes' spots
##### Fake Sale spots: This means that Reason for Cancelling Survey is any combination which contains 'Fake Sale'
##### Spots whose start date was before 2018


## CAN CANCEL:
##### Spots which cancelled more than 2 months before their Available Cancellation Date are excluded
##### Spots whose Available Cancellation Date is 2 or more months after the date of analysis
##### Months during which spots couldn't have cancelled according to the above conditions

## wo CB sets:
##### Closed Business, Sold Business and Non-payment spots are excluded. This means Reason for Cancelling Survey is any combination which contains 'Closed Business', 'Closed/Sold Business', 'Sold Business', 'Sold/Closed Business' or 'Non-payment'
## canc conf event
##### Spots for which Date Cancellation Confirmed is more than 60 days after Date Cancellation Requested

#### ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [1]:
import pandas as pd
import numpy as np
import importlib
import get_started_doing_something_variables
import prepare_for_the_models
import fit_tv_cox_models
#import read_vars_for_separate_models_for_dependent_vars
import read_a_combination_of_variables
import yaml
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display, Markdown

In [2]:
# read the yaml file with a list of parameters needed for the report
with open(r'./parameters/started_doing_something_report_parameters.yaml') as file:
    parameters = yaml.load(file, Loader=yaml.FullLoader)

date_of_analysis = parameters['date_of_analysis']
date_dir = date_of_analysis.replace('-', '_')
### name of the data directory ###
churn_based_on_behaviour_dir = parameters['churn_based_on_behaviour_dir']
### penalizer value when fitting the models ###
penalizer = parameters['penalizer']
### model type ###
model_type = parameters['model_type']
### coefficient and p values when dropping unsignificant variables ###
coefficient_limit_for_numerical_vars = parameters['coefficient_limit_for_numerical_vars']
coefficient_limit_for_cat_vars = parameters['coefficient_limit_for_cat_vars']
p_limit = parameters['p_limit']
additional_higher_p_limit = parameters['additional_higher_p_limit']
additional_lower_p_limit = parameters['additional_lower_p_limit']

# read the yaml file with data set parameters #
# data_set_name = input('Data set name: ')
data_set_name = 'ALL_spots_with_CB_cancellation_requested'
with open(r'./parameters/data_sets.yaml') as file:
    data_sets_parameters = yaml.load(file, Loader=yaml.FullLoader)
    
spots_set = data_sets_parameters[data_set_name]['spots_set']
with_wo_CB = data_sets_parameters[data_set_name]['with_wo_CB']
event_date_full_name = data_sets_parameters[data_set_name]['event_date_type']

if with_wo_CB == 'with_CB':
    with_wo_CB_boolean = True
else:
    with_wo_CB_boolean = False

if event_date_full_name == 'cancellation_requested':
    event_date = 'canc_req'
elif event_date_full_name == 'cancellation_confirmed':
    event_date = 'canc_conf'

In [3]:
##### base columns ##### 
base_cols = ['spot_id',\
            'time',\
            'event']

In [4]:
##### Model 0: all variables - prepare data for the model ####
cols_to_use = read_a_combination_of_variables.\
main(model_number=0, dir_name='combinations_of_variables_that_are_not_dependent/')
#### get behavioural variables ####
(variables_to_use_for_the_model, did_something_last_month_vars, did_something_before_and_didnt_last_month_vars) = \
get_started_doing_something_variables.main(date_of_analysis=date_of_analysis, variables_to_use_for_the_model=cols_to_use)
cols = base_cols + variables_to_use_for_the_model + \
did_something_before_and_didnt_last_month_vars + did_something_last_month_vars


#### get data for the model ###
(data, base_df, df_timeline_all_vars) = \
prepare_for_the_models.get_data_for_the_MV_Cox_model(date_of_analysis=date_of_analysis, spots_set=spots_set, \
                            with_wo_CB=with_wo_CB, event_date=event_date, columns=cols, data_dir=churn_based_on_behaviour_dir,\
                            C = 100)
    
model_numbers = \
read_a_combination_of_variables.get_a_list_of_model_numbers(dir_name='combinations_of_variables_that_are_not_dependent/')

In [5]:
for model_number in model_numbers[1:]:
    cols_to_use = read_a_combination_of_variables.\
    main(model_number=model_number, dir_name='combinations_of_variables_that_are_not_dependent/')
    
    model_name = \
    read_a_combination_of_variables.get_model_names(model_number=model_number, \
                                                    dir_name='combinations_of_variables_that_are_not_dependent/')
    display(Markdown("# Model "+ str(model_number) + ": " + model_name))

    #### get behavioural variables ####
    (variables_to_use_for_the_model, did_something_last_month_vars, did_something_before_and_didnt_last_month_vars) = \
    get_started_doing_something_variables.main(date_of_analysis=date_of_analysis, variables_to_use_for_the_model=cols_to_use)
    cols = variables_to_use_for_the_model + \
    did_something_before_and_didnt_last_month_vars + did_something_last_month_vars

    #### data for the model ###
    df_timeline = df_timeline_all_vars.copy()
    vars_that_stay = ['spot_id', 'start', 'stop', 'event']+\
    [x for x in cols if x not in base_cols]+\
    [x for x in df_timeline.columns if 'spot_category_' in x or 'metro_area_' in x]
    df_timeline.drop([x for x in df_timeline.columns if x not in vars_that_stay], axis = 1, inplace = True)

    ### variables to skip ###
    df_timeline.isnull().sum().sum() #OK
    skip_vars = list((df_timeline!=0).sum()[(df_timeline!=0).sum()==0].index)
    skip_vars

    ctv = fit_tv_cox_models.fit_the_models_and_print_summaries(df_timeline=df_timeline, base_df=base_df, \
                                                         date_of_analysis=date_of_analysis, model_type=model_type, \
                                                         variables_to_use_for_the_model=cols_to_use,\
                                                         coefficient_limit_for_numerical_vars=coefficient_limit_for_numerical_vars, \
                                                         coefficient_limit_for_cat_vars=coefficient_limit_for_cat_vars, p_limit=p_limit, \
                                                         additional_higher_p_limit=additional_higher_p_limit, \
                                                         additional_lower_p_limit=additional_lower_p_limit,\
                                                         skip_vars=skip_vars, penalizer=penalizer)

    ### save coefs and p values for p < 0.2 ###
    fit_tv_cox_models.save_results(df = ctv.summary.reset_index(), date_of_analysis = date_of_analysis, \
                                   data_dir = churn_based_on_behaviour_dir, dir_name='exports/coefficients_and_pvalues',\
                 results_name = 'coef_and_pvalues', spots_set = spots_set,\
                 with_wo_CB = with_wo_CB, event_date_type = event_date_full_name, p_limit=0.2, model_number=model_number,\
                                  model_type = model_type)

# Model 1: posts_on_facebook

## without variables with p >= 0.2

### sorted by variable name

<lifelines.CoxTimeVaryingFitter: fitted with 34817 periods, 2748 subjects, 424 events>
         event col = 'event'
         penalizer = 0.001
number of subjects = 2748
 number of periods = 34817
  number of events = 424
partial log-likelihood = -2726.83
  time fit was run = 2021-12-21 10:06:36 UTC

---
                                                     coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                  
Monthly                                              0.82       2.28       0.17             0.49             1.16                 1.63                 3.19
New.email.subscribers.allinclusive.last.month.d...  -0.04       0.96       0.02            -0.08             0.01                 0.92                 1.01
Website.Views.last.month.total.log2                 -0.15       0.86       0.04            -0.22            -0.08                 0.80                 0.92
had_catering_submissions_before_and_didnt_last_...  -0.02       0.98       0.19            -0.40             0.35                 0.67                 1.42
had_catering_submissions_last_3_months              -0.58       0.56       0.18            -0.94            -0.22                 0.39                 0.80
had_clicked_emails_before_and_didnt_last_3_months    0.36       1.43       0.13             0.11             0.61                 1.11                 1.84
had_clicked_emails_last_3_months                    -0.24       0.79       0.15            -0.54             0.06                 0.58                 1.06
had_online_orders_before_and_didnt_last_month        0.40       1.50       0.25            -0.09             0.90                 0.91                 2.46
had_online_orders_last_month                        -0.06       0.94       0.28            -0.60             0.49                 0.55                 1.63
had_other_non_contactless_menu_qr_flyer_scans_b...  -0.00       1.00       0.32            -0.63             0.62                 0.54                 1.87
had_other_non_contactless_menu_qr_flyer_scans_l...  -0.81       0.44       0.48            -1.74             0.12                 0.17                 1.13
had_posts_on_facebook_before_and_didnt_last_month    0.45       1.57       0.12             0.21             0.69                 1.23                 2.00
had_posts_on_facebook_last_month                    -0.14       0.87       0.13            -0.39             0.12                 0.67                 1.12
had_qr_code_menu_scans_before_and_didnt_last_month  -0.21       0.81       0.24            -0.69             0.27                 0.50                 1.31
had_qr_code_menu_scans_last_month                   -0.39       0.67       0.17            -0.74            -0.05                 0.48                 0.95
had_reservations_submissions_before_and_didnt_l...   0.29       1.34       0.19            -0.08             0.67                 0.92                 1.95
had_reservations_submissions_last_month             -0.23       0.80       0.25            -0.71             0.25                 0.49                 1.29
had_tickets_before_and_didnt_last_3_months          -0.46       0.63       0.12            -0.69            -0.23                 0.50                 0.79
had_tickets_last_3_months                           -0.35       0.71       0.14            -0.62            -0.07                 0.54                 0.94
had_visited_admin_before_and_didnt_last_month        0.24       1.27       0.17            -0.10             0.58                 0.91                 1.78
had_visited_admin_last_month                        -0.11       0.89       0.19            -0.48             0.25                 0.62                 1.28
had_visited_qrcode_flyers_page_before_and_didnt...  -0.34       0.71       0.22            -0.76           

# Model 2: changed_picture_or_text

...

<lifelines.CoxTimeVaryingFitter: fitted with 34817 periods, 2748 subjects, 424 events>
         event col = 'event'
         penalizer = 0.001
number of subjects = 2748
 number of periods = 34817
  number of events = 424
partial log-likelihood = -2725.32
  time fit was run = 2021-12-21 11:28:03 UTC

---
                                                     coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                  
Fb.page.likes.analysis.avg.log2                     -0.03       0.97       0.02            -0.06             0.00                 0.94                 1.00
Monthly                                              0.86       2.37       0.17             0.53             1.20                 1.70                 3.32
Website.Views.last.month.total.log2                 -0.18       0.84       0.03            -0.24            -0.11                 0.78                 0.90
had_catering_submissions_before_and_didnt_last_...  -0.04       0.96       0.19            -0.41             0.33                 0.66                 1.39
had_catering_submissions_last_3_months              -0.62       0.54       0.18            -0.97            -0.26                 0.38                 0.77
had_clicked_emails_before_and_didnt_last_3_months    0.34       1.41       0.13             0.09             0.60                 1.09                 1.81
had_clicked_emails_last_3_months                    -0.24       0.79       0.15            -0.54             0.07                 0.58                 1.07
had_online_orders_before_and_didnt_last_month        0.36       1.43       0.25            -0.14             0.85                 0.87                 2.34
had_online_orders_last_month                        -0.27       0.76       0.27            -0.80             0.25                 0.45                 1.29
had_other_non_contactless_menu_qr_flyer_scans_b...   0.04       1.04       0.32            -0.58             0.67                 0.56                 1.95
had_other_non_contactless_menu_qr_flyer_scans_l...  -0.82       0.44       0.47            -1.75             0.12                 0.17                 1.12
had_posts_liked_before_and_didnt_last_3_months       0.06       1.07       0.13            -0.18             0.31                 0.83                 1.36
had_posts_liked_last_3_months                       -0.30       0.74       0.18            -0.64             0.05                 0.52                 1.05
had_posts_on_facebook_before_and_didnt_last_month    0.37       1.44       0.14             0.10             0.63                 1.11                 1.88
had_posts_on_facebook_last_month                    -0.21       0.81       0.14            -0.48             0.06                 0.62                 1.06
had_posts_seen_before_and_didnt_last_3_months        0.25       1.28       0.18            -0.10             0.59                 0.90                 1.81
had_posts_seen_last_3_months                         0.28       1.32       0.20            -0.10             0.67                 0.90                 1.95
had_preview_page_views_before_and_didnt_last_3_...   0.37       1.45       0.19             0.01             0.74                 1.01                 2.09
had_preview_page_views_last_3_months                 0.15       1.16       0.20            -0.23             0.53                 0.79                 1.71
had_qr_code_menu_scans_before_and_didnt_last_month  -0.25       0.78       0.24            -0.73             0.23                 0.48                 1.26
had_qr_code_menu_scans_last_month                   -0.46       0.63       0.17            -0.80            -0.12                 0.45                 0.89
had_reservations_submissions_before_and_didnt_l...   0.27       1.31       0.19            -0.11           

# Model 33: New.email.subscribers.allinclusive.last.month.div10

## without variables with p >= 0.2

### sorted by variable name

<lifelines.CoxTimeVaryingFitter: fitted with 34817 periods, 2748 subjects, 424 events>
         event col = 'event'
         penalizer = 0.001
number of subjects = 2748
 number of periods = 34817
  number of events = 424
partial log-likelihood = -2723.81
  time fit was run = 2021-12-21 11:28:34 UTC

---
                                                     coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                  
Fb.page.likes.analysis.avg.log2                     -0.03       0.97       0.02            -0.06             0.00                 0.94                 1.00
Monthly                                              0.86       2.37       0.17             0.53             1.20                 1.69                 3.31
New.email.subscribers.allinclusive.last.month.d...  -0.04       0.96       0.02            -0.09             0.01                 0.92                 1.01
Website.Views.last.month.total.log2                 -0.16       0.85       0.04            -0.23            -0.09                 0.80                 0.91
had_catering_submissions_before_and_didnt_last_...  -0.04       0.96       0.19            -0.42             0.33                 0.66                 1.39
had_catering_submissions_last_3_months              -0.60       0.55       0.18            -0.95            -0.24                 0.39                 0.79
had_clicked_emails_before_and_didnt_last_3_months    0.34       1.41       0.13             0.09             0.60                 1.09                 1.82
had_clicked_emails_last_3_months                    -0.24       0.79       0.15            -0.54             0.07                 0.58                 1.07
had_online_orders_before_and_didnt_last_month        0.36       1.44       0.25            -0.13             0.85                 0.88                 2.35
had_online_orders_last_month                        -0.20       0.82       0.27            -0.73             0.34                 0.48                 1.40
had_other_non_contactless_menu_qr_flyer_scans_b...   0.05       1.05       0.32            -0.57             0.68                 0.56                 1.97
had_other_non_contactless_menu_qr_flyer_scans_l...  -0.81       0.45       0.48            -1.74             0.13                 0.18                 1.13
had_posts_liked_before_and_didnt_last_3_months       0.06       1.06       0.13            -0.19             0.30                 0.83                 1.35
had_posts_liked_last_3_months                       -0.30       0.74       0.18            -0.65             0.04                 0.52                 1.04
had_posts_on_facebook_before_and_didnt_last_month    0.37       1.44       0.14             0.10             0.63                 1.10                 1.88
had_posts_on_facebook_last_month                    -0.22       0.81       0.14            -0.49             0.05                 0.61                 1.06
had_posts_seen_before_and_didnt_last_3_months        0.25       1.28       0.18            -0.10             0.59                 0.90                 1.81
had_posts_seen_last_3_months                         0.28       1.32       0.20            -0.11             0.66                 0.90                 1.94
had_preview_page_views_before_and_didnt_last_3_...   0.37       1.45       0.19             0.01             0.73                 1.01                 2.08
had_preview_page_views_last_3_months                 0.16       1.17       0.20            -0.23             0.54                 0.80                 1.72
had_qr_code_menu_scans_before_and_didnt_last_month  -0.24       0.79       0.24            -0.72             0.24                 0.49                 1.27
had_qr_code_menu_scans_last_month                   -0.44       0.64       0.17            -0.78           